In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from utils import draw_tracking_result, save_tracking_results, load_detections
from tracks import track_iou, create_similarity_matrix, associate_detections_to_tracks, update_tracks, initialize_new_track, look_forward, init_model
from KalmanFilter import KalmanFilter

###  Loading det object instances

In [2]:
det_file_path = '../data/det/det.txt'
gt_gile_path = '../data/gt/gt.txt'
detections_df = load_detections(det_file_path)
detections_df.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1


In [3]:
# Show min bb_width and bb_height
print('Min bb_width: ', detections_df['bb_width'].min())
print('Min bb_height: ', detections_df['bb_height'].min())

Min bb_width:  26.0
Min bb_height:  59.0


In [4]:
# Convert conf column to float
detections_df['conf'] = detections_df['conf'].astype(float)

#### Interface (main loop)

In [5]:
'''# Main loop
image_frames_path = '../data/img1/' 

# Define properties for the output video
fps = 40
frame_size = (1920, 1080) # Adjust this to match the size of your frames
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, frame_size)
delay = int(1000/fps)

# Initialize tracking variables
tracks_history = {}
frame_number = 1
CONF_THRESH = 28.0
sigma = 0.1
nb_step = 10

init_model()

tracking_file = 'track.txt'
if os.path.isfile(tracking_file):
    open(tracking_file, 'w').close()
else:
    open(tracking_file, 'x')

# Process each image frame
for filename in sorted(os.listdir(image_frames_path)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(image_frames_path, filename)
        frame = cv2.imread(frame_path)

        # Looking forward
        tracks_history = look_forward(
            detections_df,
            tracks_history,
            frame_number,
            frame,
            image_frames_path,
            nb_step=nb_step,
            sigma=sigma,
            conf_threshold=CONF_THRESH
        )
        
        # Draw tracking
        frame_with_tracking = draw_tracking_result(frame, tracks_history[frame_number])
        # Save the tracking results
        save_tracking_results(tracks_history[frame_number], tracking_file, frame_number)

        # Show the frame
        cv2.imshow('Tracking', frame_with_tracking)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
        out.write(frame_with_tracking)

        frame_number += 1

out.release()
cv2.destroyAllWindows()'''

'# Main loop\nimage_frames_path = \'../data/img1/\' \n\n# Define properties for the output video\nfps = 40\nframe_size = (1920, 1080) # Adjust this to match the size of your frames\nfourcc = cv2.VideoWriter_fourcc(*\'mp4v\')  # Use \'mp4v\' codec\nout = cv2.VideoWriter(\'output_with_tracking.mp4\', fourcc, fps, frame_size)\ndelay = int(1000/fps)\n\n# Initialize tracking variables\ntracks_history = {}\nframe_number = 1\nCONF_THRESH = 28.0\nsigma = 0.1\nnb_step = 10\n\ninit_model()\n\ntracking_file = \'track.txt\'\nif os.path.isfile(tracking_file):\n    open(tracking_file, \'w\').close()\nelse:\n    open(tracking_file, \'x\')\n\n# Process each image frame\nfor filename in sorted(os.listdir(image_frames_path)):\n    if filename.endswith(".jpg"):\n        frame_path = os.path.join(image_frames_path, filename)\n        frame = cv2.imread(frame_path)\n\n        # Looking forward\n        tracks_history = look_forward(\n            detections_df,\n            tracks_history,\n            fram

Removing noise

In [6]:
import pandas as pd

# Define minimum box width and height thresholds
MIN_WIDTH_THRESHOLD = 30
MIN_HEIGHT_THRESHOLD = 60

# Filter out detections with boxes smaller than the threshold
filtered_detections_df = detections_df[
    (detections_df['bb_width'] >= MIN_WIDTH_THRESHOLD) &
    (detections_df['bb_height'] >= MIN_HEIGHT_THRESHOLD)
]

In [7]:
detections_df.to_csv('../data/det/preprocessed_det.txt', sep=',', index=False, header=False)